## Import libraries

In [1]:
import os
import sys

# Add library path to Python path
sys.path.append(os.path.abspath('..'))

# Now import your modules
from library import libStopsPoints, libConnections, libHex
import zipfile
import os
import time
import pymongo as pym
import pandas as pd
import folium
import numpy as np
import requests
import numba

import math
import geopy
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from geopy.distance import geodesic,great_circle
from folium.plugins import FastMarkerCluster
from datetime import datetime
from geopy.distance import geodesic,great_circle
from pymongo import MongoClient

from pathlib import Path
import geopandas as gpd
import geojson
from IPython.core.display import display, HTML

from library import libStopsPoints, libConnections, libHex
 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12564\3252682998.py:31: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Setup input parameters, paths and urls

In [45]:
# scenario name and settings
city = 'Caltanissetta'
scenario_name = 'DRT_SF' # basecase = Fixed PT

# file paths and addresses to be provided:
directorySHP = f"./shp/{city}/"
directoryGTFS = f"./gtfs/{city}/{scenario_name}/"

# population data
popCollectionName = "pop"
popField = "pop"

# required parameters
# the date must be in the interval of validity of the gtfs files
# check it in the "calendar.txt" and "calendar_dates.txt" files inside the gtfs zip files.
days = ['20250407']
day_names = [ "monday"]

# parameters of walking distance
timeWalk = 15 * 60  # seconds
velocityWalk = 1.31  # m/s ***4.71km/h***
distanceS = timeWalk * velocityWalk

# Parameters that define the resolution and extention of tesselletion and the maximum of the walking time
# grid step of the hexagonal tesselletion in kilometers
gridEdge = 0.4

#boundaries of the study area
bbox = [14.010000, 37.450000, 14.100000, 37.515000] #EPSG:4326

# Set it True if opportunities (POIs, etc.) are discrete points
# Set it False if opportunities are attributes of polygonal shapes
oppurtunities_as_discrete_points = False

#address of databases to be created in MongoDB
gtfs_prep_toggle = True
urlMongoDb = "mongodb://localhost:27017/"  # url of the mongodb database
urlMongoDbPop = "mongodb://localhost:27017/" # url of the mongodb database for population
urlServerOsrm = 'http://localhost:5000/' # url of the osrm server of the city

client = MongoClient(urlMongoDb)
gtfsDB = client[str('pta-' + city + '-' + scenario_name)]
popDbName = str('pta-' + city + '-' + scenario_name)

# Set check4stops = False if cells / hexagones should be included that do not have stops within.
# Set check4stops = False for preprocessing prior to dynamic mode to gtfs convertion
# Set check4stops = True for citychrone accessibility analysis
check4stops = False

## Create Database in MongoDB

In [11]:
# 1. load population data
print("loading population data...")
shpPath = str(directorySHP + 'pop.shp')
geojsonPath = str(directorySHP + 'pop.geojson')
shapefile = gpd.read_file(shpPath)
shapefile.to_file(geojsonPath, driver='GeoJSON')
with open(geojsonPath, encoding="utf8") as f:
    gj = geojson.load(f)
features = gj['features']
gtfsDB["pop"].drop()
gtfsDB["pop"].insert_many(features)

# 2. load GTFS data
print("loading GTFS...")
listOfFile = ['stops.txt', 'routes.txt', 'trips.txt',
              'calendar.txt', 'calendar_dates.txt', 'stop_times.txt']  
libStopsPoints.loadGtfsFile(gtfsDB, directoryGTFS, city, listOfFile)

# 3. Process GTFS creating "connections" representing trips between stops
# extract connection information between stops for specified dates and days of the week
# handles frequency-based services, and stores everything in MongoDB for transit network analysis and schedule validation
print("creating connections from GTFS...")
libConnections.readConnections(gtfsDB, city, directoryGTFS, days[0], day_names[0], overwrite=True)

# 4. Remove stops with no connections and add to each stop the pos field
libStopsPoints.removingStopsNoConnections(gtfsDB, city)
libStopsPoints.setPosField(gtfsDB, city)

libConnections.updateConnectionsStopName(gtfsDB, city)


loading population data...
loading GTFS...
removing stops  of  Caltanissetta
removing routes  of  Caltanissetta
removing trips  of  Caltanissetta
removing calendar  of  Caltanissetta
removing calendar_dates  of  Caltanissetta
removing stop_times  of  Caltanissetta
gtfs_DRT_DRT_SF.zip
stops.txt -> (174,174)
routes.txt -> (118816,118816)
trips.txt -> (118816,118816)
calendar.txt -> (1,1)
stop_times.txt -> (237632,237632)
creating connections from GTFS...
number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs_DRT_DRT_SF.zip 	 total number of active service (in calendar.txt): 1
number of different service_id: 1


file: gtfs_DRT_DRT_SF.zip 	 total number of active service (in calendar_dates.txt): 1
number of different service_id: 1 total number of active services found: 1
number of trips 118000
 gtfs_DRT_DRT_SF.zip
inserting to DB....18816, err 0, err_start 0, err_start_after 0
tot connections 118816
connections dele

## Step 1 - Study Area Tessellation

### Compute the box that include all stops
The edge of such box are enlarged by distanceS.

In [4]:
stopsList = libStopsPoints.returnStopsList(gtfsDB, city)
display(HTML('<h1>All stops of the public transport present in the gtfs files</h1>'))

# comment bbox = ... if you want to use the boundaries set as input parameter
#bbox = libStopsPoints.boundingBoxStops(stopsList)
libStopsPoints.mapStops(bbox, stopsList)

tot stop 174  stop error : 0


### Tassel the study area with hexagons.

In [5]:
# Standardized spatial framework for analyzing transit accessibility and coverage across the study area

# 1. Create a uniform hexagonal grid over a geographic area,
# optionally filtering to include only hexagons near transit stops
# Each hexagon contains its geometric properties and location data
hexBin, pointBin = libHex.hexagonalGrid(bbox, gridEdge, gtfsDB['stops'], distanceS, city,check4stops)

# 2. Store the hexagonal grid in MongoDB with appropriate indexes for spatial queries.
libHex.insertPoints(pointBin, city, gtfsDB) 

### Evaluation of hexagons' walking accessibility from stops

In [11]:
# 3. Using OSRM routing engine,calculate actual walking times from each transit stop to nearby hexagon centroids,
# An hexagon is "served" if it is within the walking time threshold.
libHex.pointsServed(gtfsDB, stopsList, urlServerOsrm, distanceS, timeWalk, city)
print("\nNumber of hexagons: {0}".format(gtfsDB['points'].count_documents({'served':True, 'city':city})))

# 4. Assigns sequential position IDs to all the served hexagons
libHex.settingHexsPos(gtfsDB, city)

 tot 250,100%, removed 0  251
Number of hexagons: 251


### Opportunities within Hexagons (POIs, population, ...) 


In [13]:
def setHexsPop(gtfsDB, popCol, namePopField, city):
    tot = gtfsDB['points'].count_documents({'city': city})
    count = 0
    totPop = 0

    for hexagon in gtfsDB['points'].find({'city': city}):
        shapelyHex = Polygon(hexagon['hex']['coordinates'][0])
        hexagon['hex']['properties'] = {'pop': 0}
        findJson = {'geometry': {'$geoIntersects': {'$geometry': hexagon['hex']}}}

        for box in popCol.find(findJson):
            # Check if the geometry is a Point or Polygon
            if box['geometry']['type'] == 'Point':
                shapelyPoint = Point(box['geometry']['coordinates'])
                if shapelyHex.contains(shapelyPoint):
                    hexagon['hex']['properties']['pop'] += box['properties'][namePopField]
            elif box['geometry']['type'] == 'Polygon':
                shapelyBox = Polygon(box['geometry']['coordinates'][0])
                areaInter = shapelyBox.intersection(shapelyHex).area
       
                
                popHexBox = box['properties'][namePopField] * areaInter / shapelyBox.area
                hexagon['hex']['properties']['pop'] += popHexBox
            else:
                print("Unexpected geometry type:", box['geometry']['type'])
                continue

        count += 1
        totPop += hexagon['hex']['properties']['pop']
        gtfsDB['points'].update_one({'_id': hexagon['_id']}, {'$set': {'pop': hexagon['hex']['properties']['pop']}})
        print('{0:.1f}% , tot population: {1:.0f}, current hex: {2:.0f}'.format(100. * count / tot, totPop, hexagon['hex']['properties']['pop']), end="\r")

if urlMongoDbPop != "" and popCollectionName != "":
    clientPop = pym.MongoClient(urlMongoDbPop)
    popDb = clientPop[popDbName]
    popCollection = popDb[popCollectionName]
    setHexsPop(gtfsDB, popCollection, popField, city)
else:
    print("Population NOT INSERTED!")

# based on what data type of opportunities you have (discrete points vs. polygons) run different commands
if oppurtunities_as_discrete_points:
    res = gtfsDB['points'].update_many({'pop': {'$exists': False}}, {'$set': {'pop': 0}})
    print("n° of matched hexagons with POIs: {0} \n not matched: {1} (set to zero)".format(
        gtfsDB['points'].count_documents({'pop': {'$exists': True}}), res.modified_count))
else:
    if urlMongoDbPop != "" and popCollectionName != "":
        clientPop = pym.MongoClient(urlMongoDbPop)
        popDb = clientPop[popDbName]
        popCollection = popDb[popCollectionName]
        libHex.setHexsPop(gtfsDB, popCollection, popField, city)
    else:
        print("Population NOT INSERTED!")

    res = gtfsDB['points'].update_many({'pop':{'$exists':False}}, {'$set':{'pop':0}})
    print("n° of matched hexagons with population Polygons: {0} \n \
    not matched: {1} (setted to zero)".format(
        gtfsDB['points'].count_documents({'pop':{'$exists':True}}), res.modified_count))

n° of matched hexagons with population Polygons: 251 
     not matched: 0 (setted to zero)


## Step 2 - Compute Accessibility metrics

### Set parameters for accessibility analysis

In [ ]:
import imp
from library import libAccessibility, icsa
imp.reload(libAccessibility)
from library.icsa import computeAccessibilities
imp.reload(icsa)

# list of starting time for computing the isochrones
timeList = [8,12,16,20]
hStart = timeList[0]*3600

listAccessibility = ['accessibility30min']
tmax = 1800 #30min
timestamps = ['avg']

#listAccessibility --> choose among accessibility60min', 'accessibility30min' 'accessibility20min']

### Compute walking time between stops and centroids

In [23]:
# Create a "neighborhood" relationship map by finding which stops and centroids (hexagons)
#                   are accessible to each other within a specified walking time threshold
# It uses the OSRM routing service to calculate actual walking times
libStopsPoints.computeNeigh(gtfsDB, urlServerOsrm, distanceS, timeWalk,  city)
# These relationshpis are stored in the database

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


### List of connections

In [24]:
# arrayCC = libConnections.makeArrayConnections(gtfsDB, hStart, city)
arrayCC_day1 = libConnections.makeArrayConnections(gtfsDB, hStart, city, day=days[0]) #day='20250407'


start making connections array
done recover all cc 118816
converted
Num of connection 118816


### List of list of the points and stops neighbors

In [25]:
# Convert the neighbor data from MongoDB format into numeric array representation, for faster computation in accessibility analysis algorithms
# The arrays make it possible to quickly access walking times between any two locations in the network without having to query the database repeatedly.
arraySP = libStopsPoints.listPointsStopsN(gtfsDB, city)

### * Compute Accessibility *

In [26]:
computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    
    pass
for timeStart in timeList:
    timeStart *= 3600
    hEnd  = timeStart+6600 #consider only connection that start at timeStart and end before next timeStart in the list timeList
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
   
    computeAccessibilities(
    city, timeStart,hEnd, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day=days[0], 
    max_travel_time= tmax,
    listAccessibility=listAccessibility
)

    

Time Isochrone Start: 12.00min_20250407: 2270.7, time to finish: 0.0h, 0.0mm


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

Time Isochrone Start: 16.00min_20250407: 2270.7, time to finish: 0.0h, 0.0mm


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

Time Isochrone Start: 20.00min_20250407: 2270.7, time to finish: 0.0h, 0.0mm


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymo

### Compute Average accessibility

In [28]:
from library.libStopsPoints import computeAverage

# Create all combinations of accessibility metrics and days
for accessibility in listAccessibility:
    for day in days:
        acc_metric_name = f"{accessibility}_{day}"
        computeAverage([acc_metric_name], gtfsDB, city)

### Accessibility Map

In [43]:
import folium
import branca.colormap as cm
from IPython.display import display, HTML
import numpy as np
import math

def create_accessibility_map_with_header(gtfsDB, city, field1='accessibility30min_20250331', timestamp='32400', 
                                       custom_title="Accessibility - 2025/03/31 9:00", legend_time_text="9:00"):
    """
    Create an accessibility map with a header title and top-right colorbar
    
    Parameters:
    -----------
    gtfsDB : MongoDB connection
        MongoDB database connection
    city : str
        City name
    field1 : str
        Primary field name for the accessibility score
    timestamp : str
        Timestamp key within the field1 dictionary
    custom_title : str
        Custom title to display above the map
    legend_time_text : str
        Text to display in the legend for the time (e.g., "9:00" or "Daily Average")
        
    Returns:
    --------
    HTML object
        HTML with title and map
    """
    # Get the data from MongoDB
    points_data = list(gtfsDB['points'].find({'city': city}))
    
    if not points_data:
        return HTML("<p>No data found for the specified city</p>")

    # Extract scores
    scores = []
    for point in points_data:
        try:
            if field1 in point and isinstance(point[field1], dict) and timestamp in point[field1]:
                scores.append(point[field1][timestamp])
        except (KeyError, TypeError):
            continue
    
    if not scores:
        return HTML(f"<p>No scores found for field {field1} and timestamp {timestamp}</p>")
    
    # Filter out zeros and negative values for log scale
    positive_scores = [s for s in scores if s > 0]
    if not positive_scores:
        return HTML("<p>No positive scores found for logarithmic scale</p>")
    
    # Calculate statistical values
    min_score = min(positive_scores)
    max_score = max(positive_scores)
    
    print(f"Score statistics:")
    print(f"Min: {min_score:.2f}, Max: {max_score:.2f}")
    print(f"Total points: {len(scores)}, Positive points: {len(positive_scores)}")
    
    # Find center of the map
    center_lat, center_lon = 0, 0
    count = 0
    
    for point in points_data:
        try:
            point_coords = point['point']['coordinates']
            center_lon += point_coords[0]
            center_lat += point_coords[1]
            count += 1
        except (KeyError, IndexError):
            continue
    
    if count > 0:
        center_lat /= count
        center_lon /= count
    else:
        # Default center if no points have valid coordinates
        center_lat, center_lon = 0, 0
    
    # Create map with OpenStreetMap as base layer
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11, 
                  tiles='OpenStreetMap')
    
    # Create a LogNorm-like transformation for scores
    # Based on matplotlib's LogNorm approach
    vmin = 1.0  # Minimum for log scale
    vmax = 60000.0  # Max set to 10^5 for consistency
    
    # Function to transform scores using log scale
    def log_transform(score):
        if score <= 0:
            return 0
        normalized = np.clip((np.log10(score) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin)), 0, 1)
        return normalized
    
    # Create a list of hexagons with their scores
    hexagons = []
    for point in points_data:
        try:
            if field1 in point and isinstance(point[field1], dict) and timestamp in point[field1]:
                score = point[field1][timestamp]
                
                # Skip non-positive scores for log scale
                if score <= 0 or 'hex' not in point or 'coordinates' not in point['hex']:
                    continue
                
                # Add to list for processing
                hexagons.append({
                    'coordinates': point['hex']['coordinates'][0],
                    'score': score
                })
        except (KeyError, TypeError, ValueError):
            continue
    
    # Sort hexagons by score for proper layering
    hexagons.sort(key=lambda x: x['score'])
    
    # Use Blues colormap similar to matplotlib example
    # Colors from Blues colormap
    blues_colors = [
        '#f7fbff', '#deebf7', '#c6dbef', '#9ecae1', '#6baed6', 
        '#4292c6', '#2171b5', '#08519c', '#08306b'
    ]
    
    # Add all hexagons to the map
    for hexagon in hexagons:
        # Get score and coordinates
        score = hexagon['score']
        coords = hexagon['coordinates']
        
        # Transform score using log scale
        normalized_score = log_transform(score) #score / vmax
        print(normalized_score)
        
        # Get color based on normalized score
        color_idx = min(int(normalized_score * (len(blues_colors) - 1)), len(blues_colors) - 1)
        color = blues_colors[color_idx]
        
        # Convert to folium coordinates
        folium_coords = [[coord[1], coord[0]] for coord in coords]
        
        # Add polygon to map
        folium.Polygon(
            locations=folium_coords,
            color='white',
            weight=0.5,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            tooltip=f"Score: {score:.1f}"
        ).add_to(m)
    
    # Add boundary if available
    try:
        boundary_data = list(gtfsDB['boundary'].find({'city': city}))
        if boundary_data:
            for boundary in boundary_data:
                coords = boundary['geometry']['coordinates'][0]
                folium_coords = [[coord[1], coord[0]] for coord in coords]
                folium.Polygon(
                    locations=folium_coords,
                    color='black',
                    weight=2,
                    fill=False,
                ).add_to(m)
    except Exception as e:
        print(f"Could not add boundary: {e}")
    
    # Create custom colorbar HTML with explicit markers - modified to use the legend_time_text parameter
    colorbar_html = f"""
    <div style="position: absolute; top: 10px; right: 10px; background: white; padding: 5px; border-radius: 5px; z-index: 100000;">
        <div style="width: 250px; height: 25px; background: linear-gradient(to right, 
                #f7fbff, #deebf7, #c6dbef, #9ecae1, #6baed6, 
                #4292c6, #2171b5, #08519c, #08306b);
            position: relative;">
            <div style="position: absolute; top: 25px; left: 0; width: 1px; height: 5px; background: black;"></div>
            <div style="position: absolute; top: 25px; left: 20%; width: 1px; height: 5px; background: black;"></div>
            <div style="position: absolute; top: 25px; left: 40%; width: 1px; height: 5px; background: black;"></div>
            <div style="position: absolute; top: 25px; left: 60%; width: 1px; height: 5px; background: black;"></div>
            <div style="position: absolute; top: 25px; left: 80%; width: 1px; height: 5px; background: black;"></div>
            <div style="position: absolute; top: 25px; right: 0; width: 1px; height: 5px; background: black;"></div>
            <div style="position: absolute; top: 32px; left: 0%; transform: translateX(-50%); font-size: 11px;">10<sup>0</sup></div>
            <div style="position: absolute; top: 32px; left: 20%; transform: translateX(-50%); font-size: 11px;">10<sup>1</sup></div>
            <div style="position: absolute; top: 32px; left: 40%; transform: translateX(-50%); font-size: 11px;">10<sup>2</sup></div>
            <div style="position: absolute; top: 32px; left: 60%; transform: translateX(-50%); font-size: 11px;">10<sup>3</sup></div>
            <div style="position: absolute; top: 32px; left: 80%; transform: translateX(-50%); font-size: 11px;">10<sup>4</sup></div>
            <div style="position: absolute; top: 32px; left: 100%; transform: translateX(-50%); font-size: 11px;">10<sup>5</sup></div>
        </div>
        <div style="text-align: center; margin-top: 20px; font-size: 11px;">Number of reachable POIs {legend_time_text}</div>
    </div>
    """
    
    # Add north arrow
    north_arrow_html = """
    <div style="position: absolute; bottom: 30px; right: 10px; background: white; padding: 5px; border-radius: 5px; z-index: 1000;">
        <div style="font-size: 20px; text-align: center;">⬆</div>
        <div style="font-size: 10px; text-align: center;">N</div>
    </div>
    """
    
    # Add both elements to the map
    m.get_root().html.add_child(folium.Element(colorbar_html))
    m.get_root().html.add_child(folium.Element(north_arrow_html))
    
    # Get the HTML representation of the map
    map_html = m._repr_html_()
    
    # Create the final HTML with the centered title above the map
    final_html = f"""
    <div style="text-align: center; width: 100%;">
        <h2 style="margin-bottom: 10px;">{custom_title}</h2>
        <div style="width: 100%;">
            {map_html}
        </div>
    </div>
    """
    
    return HTML(final_html)

# __MAIN__

# Format the hStart time for the legend
hours = hStart // 3600
minutes = (hStart % 3600) // 60
hStart_formatted = f"{hours}:{minutes:02d}"
legend_base_text = f"at {hStart_formatted}"  # e.g., "at 8:00"

# For each combination of parameters
for accessibility in listAccessibility:
    for day in days:
        for timestamp in timestamps:
            # Format field name for MongoDB query
            field_name = f"{accessibility}_{day}"
            
            # Format readable date for title
            try:
                date_obj = datetime.strptime(day, "%Y%m%d")
                formatted_date = date_obj.strftime("%Y/%m/%d")
            except ValueError:
                formatted_date = day
            
            # Determine legend text based on timestamp
            if timestamp == 'avg':
                legend_time_text = "(Daily Average)"
            else:
                legend_time_text = legend_base_text  # Use the hStart formatted time
            
            # Generate title with the hStart time or "Daily Average"
            if timestamp == 'avg':
                custom_title = f"Accessibility - {formatted_date} (Daily Average)"
            else:
                custom_title = f"Accessibility - {formatted_date} {hStart_formatted}"
            
            # Create and display map using the existing function
            html_output = create_accessibility_map_with_header(
                gtfsDB, 
                city, 
                field1=field_name, 
                timestamp=timestamp,
                custom_title=custom_title
            )
            display(html_output)

Score statistics:
Min: 494.61, Max: 79113.52
Total points: 251, Positive points: 251
0.5638720640372901
0.5638720640372901
0.5638720640372901
0.5906925221012546
0.600890357326943
0.600890357326943
0.600890357326943
0.600890357326943
0.6206680659813887
0.6280869944023787
0.6280869944023787
0.6337155070551052
0.6337155070551052
0.6337155070551052
0.6337155070551052
0.6337155070551052
0.6337155070551052
0.6338358633043223
0.6379910622840861
0.6388243892000478
0.6510469176968328
0.6554488662454481
0.6554488662454481
0.6554488662454481
0.6554488662454481
0.6580250375898851
0.6580250375898851
0.672647616919055
0.672647616919055
0.6729618647161222
0.6729618647161222
0.6731910323035379
0.6893305553030916
0.6893305553030916
0.6893305553030916
0.6958287472345056
0.6986309932291318
0.6986309932291318
0.7022876034935378
0.7023990778433615
0.7033035453390927
0.7033035453390927
0.70704866765676
0.7072773467146918
0.7075180213136112
0.7077824976481153
0.7078462446737999
0.7078462446737999
0.707846244